In [ ]:
%load_ext autoreload
%autoreload 2


from pathlib import Path
import json
import random
import os
from src import functions_openai2, metadata, objects
from setup import output_struct
import datetime


client = functions_openai2.get_openai_client()

In [ ]:
with open('or_bt1_t1_data.json', 'r') as f:
    data = json.load(f)

In [ ]:
# SETUP SETTINGS
prompt_name = 'zero_shot'
prompt_file = 'setup/prompts.yaml'

output_structure = output_struct.TopicAI

llm_seed = 42
temperature = 0.0

model = 'gpt-4o-2024-08-06'

In [ ]:
prompt = objects.SysUserPrompt()
prompt.load_from_file(prompt_file, prompt_name)


In [ ]:
result =functions_openai2.loop_prompt(
    data,
    prompt,
    client=client,
    response_format=output_structure,
    seed=llm_seed,
    temperature=temperature,
    model=model,
)


In [ ]:
gold, pred = functions_openai2.gold_prediction_lists(
    result, 'topic_ai'
)

In [ ]:
final_corpus_path = Path('/home//.../final_corpus')
corpus_files = list(final_corpus_path.glob('*.json'))

for res in result:
    filestem = res['filename']
    # Find the corresponding corpus file
    corpus_file = next((f for f in corpus_files if f.stem == filestem), None)
    if corpus_file:
        with open(corpus_file, 'r') as f:
            document = json.load(f)
        document['GPT4o'] = functions_openai2.get_prediction(
            res, 'topic_ai'
        )
        with open(corpus_file, 'w') as f:
            json.dump(document, f, indent=4, ensure_ascii=False)